In [ ]:
# default_exp market-maker

# module issu de 01_market-maker

> module du 2ème notebook de test

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
say_hello('Sylvain')

'Hello Sylvain!'

In [ ]:
assert say_hello('Hedi') == 'Hello Hedi!'